# Backend System Examples

This notebook demonstrates the usage of rompy's backend system for running models, processing outputs, and orchestrating complete workflows.

The backend system provides three types of components:
- **Run Backends**: Execute models in different environments
- **Postprocessors**: Process model outputs after execution
- **Pipeline Backends**: Orchestrate complete workflows

In [2]:
import logging
from datetime import datetime
from pathlib import Path
import tempfile

from rompy.model import ModelRun
from rompy.core.config import BaseConfig
from rompy.core.time import TimeRange
from rompy.core.logging import get_logger

# Configure logging to see backend activity
logger = get_logger(__name__)

## Setup: Create a Basic Model Run

First, let's create a basic model run that we can use to demonstrate the different backends.

In [3]:
# Create a temporary directory for outputs
output_dir = tempfile.mkdtemp(prefix="rompy_backend_demo_")
print(f"Output directory: {output_dir}")

# Create a model run
model = ModelRun(
    run_id="backend_demo",
    period=TimeRange(
        start=datetime(2023, 1, 1, 0),
        end=datetime(2023, 1, 1, 6),
        interval="1H"
    ),
    output_dir=output_dir,
    config=BaseConfig(arg1="demo", arg2="backend_test"),
    delete_existing=True
)

print(f"Model run created: {model.run_id}")
print(f"Period: {model.period.start} to {model.period.end}")

Output directory: /tmp/rompy_backend_demo_45ohtrfj
Model run created: backend_demo
Period: 2023-01-01 00:00:00 to 2023-01-01 06:00:00


## Discovering Available Backends

Let's see what backends are available in the current environment.

In [4]:
from rompy.model import RUN_BACKENDS, POSTPROCESSORS, PIPELINE_BACKENDS

print("Available Run Backends:")
for name, backend_class in RUN_BACKENDS.items():
    print(f"  - {name}: {backend_class.__name__}")

print("\nAvailable Postprocessors:")
for name, processor_class in POSTPROCESSORS.items():
    print(f"  - {name}: {processor_class.__name__}")

print("\nAvailable Pipeline Backends:")
for name, pipeline_class in PIPELINE_BACKENDS.items():
    print(f"  - {name}: {pipeline_class.__name__}")

Available Run Backends:
  - docker: DockerRunBackend
  - local: LocalRunBackend

Available Postprocessors:
  - noop: NoopPostprocessor

Available Pipeline Backends:
  - local: LocalPipelineBackend


## Run Backends

Run backends execute the model in different environments. Let's demonstrate the available backends.

### Local Backend

The local backend runs the model directly on the current system.

In [5]:
# Run with local backend (default)
print("Running with local backend...")
success = model.run(backend="local")

print(f"Run successful: {success}")

# Check what files were created
output_path = Path(output_dir) / model.run_id
if output_path.exists():
    files = list(output_path.rglob("*"))
    print(f"\nFiles created: {len(files)}")
    for file in files[:5]:  # Show first 5 files
        print(f"  - {file.relative_to(output_path)}")
    if len(files) > 5:
        print(f"  ... and {len(files) - 5} more")

Running with local backend...


TypeError: Backend must be a BackendConfig instance, got str

### Docker Backend (if available)

The Docker backend runs the model inside a Docker container. This requires Docker to be installed and running.

In [ ]:
# Check if Docker backend is available
if "docker" in RUN_BACKENDS:
    print("Docker backend is available!")
    
    # Note: This example shows the syntax but may fail if Docker is not available
    # or if the required image is not present
    try:
        print("\nAttempting to run with Docker backend...")
        print("(This may fail if Docker is not available or configured)")
        
        success = model.run(
            backend="docker",
            image="ubuntu:20.04",  # Simple base image for demonstration
            executable="/bin/echo",  # Simple command that will succeed
            env_vars={"TEST_VAR": "backend_demo"}
        )
        
        print(f"Docker run successful: {success}")
        
    except Exception as e:
        print(f"Docker run failed (expected if Docker not available): {e}")
else:
    print("Docker backend is not available in this environment.")

## Postprocessors

Postprocessors handle model outputs after execution.

### No-op Postprocessor

The no-op postprocessor is a placeholder that performs no operations.

In [ ]:
# Use the no-op postprocessor
print("Running no-op postprocessor...")
results = model.postprocess(processor="noop")

print(f"Postprocessing results: {results}")

### Custom Postprocessor Example

Let's create a simple custom postprocessor to demonstrate how to extend the system.

In [ ]:
import os
from typing import Dict, Any

class FileCountPostprocessor:
    """A simple postprocessor that counts files in the output directory."""
    
    def process(self, model_run, **kwargs) -> Dict[str, Any]:
        """Count files in the model output directory."""
        output_path = Path(model_run.output_dir) / model_run.run_id
        
        if not output_path.exists():
            return {
                "success": False,
                "message": "Output directory does not exist",
                "file_count": 0
            }
        
        # Count files
        file_count = sum(1 for f in output_path.rglob("*") if f.is_file())
        total_size = sum(f.stat().st_size for f in output_path.rglob("*") if f.is_file())
        
        return {
            "success": True,
            "message": f"Found {file_count} files",
            "file_count": file_count,
            "total_size_bytes": total_size,
            "output_directory": str(output_path)
        }

# Use our custom postprocessor
print("Running custom file count postprocessor...")
custom_processor = FileCountPostprocessor()
results = custom_processor.process(model)

print(f"Custom postprocessing results:")
for key, value in results.items():
    print(f"  {key}: {value}")

## Pipeline Backends

Pipeline backends orchestrate the complete workflow from model generation through execution to postprocessing.

### Local Pipeline

The local pipeline backend executes the complete workflow locally.

In [ ]:
# Create a fresh model run for the pipeline demo
pipeline_model = ModelRun(
    run_id="pipeline_demo",
    period=TimeRange(
        start=datetime(2023, 1, 1, 0),
        end=datetime(2023, 1, 1, 3),
        interval="1H"
    ),
    output_dir=output_dir,
    config=BaseConfig(arg1="pipeline", arg2="demo"),
    delete_existing=True
)

# Run the complete pipeline
print("Running complete pipeline...")
results = pipeline_model.pipeline(
    pipeline_backend="local",
    run_backend="local",
    processor="noop"
)

print(f"\nPipeline results:")
for key, value in results.items():
    print(f"  {key}: {value}")

## Error Handling

The backend system provides clear error messages when backends are not found.

In [ ]:
# Try to use a non-existent run backend
try:
    model.run(backend="nonexistent")
except ValueError as e:
    print(f"Run backend error: {e}")

# Try to use a non-existent postprocessor
try:
    model.postprocess(processor="invalid")
except ValueError as e:
    print(f"Postprocessor error: {e}")

# Try to use a non-existent pipeline backend
try:
    model.pipeline(pipeline_backend="unknown")
except ValueError as e:
    print(f"Pipeline backend error: {e}")

## Complete Workflow Example

Let's put it all together with a complete workflow that demonstrates the full capability of the backend system.

In [ ]:
# Create a comprehensive workflow
workflow_model = ModelRun(
    run_id="complete_workflow",
    period=TimeRange(
        start=datetime(2023, 1, 1, 0),
        end=datetime(2023, 1, 1, 12),
        interval="3H"
    ),
    output_dir=output_dir,
    config=BaseConfig(
        arg1="complete",
        arg2="workflow",
        description="Demonstration of complete backend workflow"
    ),
    delete_existing=True
)

print("=== Complete Workflow Demo ===")
print(f"Model: {workflow_model.run_id}")
print(f"Period: {workflow_model.period}")
print(f"Output: {workflow_model.output_dir}")

# Step 1: Generate input files
print("\n1. Generating input files...")
workflow_model.generate()
print("   ✓ Input files generated")

# Step 2: Run the model
print("\n2. Running model...")
run_success = workflow_model.run(backend="local")
print(f"   ✓ Model run: {'Success' if run_success else 'Failed'}")

# Step 3: Process outputs
print("\n3. Processing outputs...")
process_results = workflow_model.postprocess(processor="noop")
print(f"   ✓ Postprocessing: {process_results['message']}")

# Step 4: Complete pipeline (alternative approach)
print("\n4. Alternative: Complete pipeline in one call...")
pipeline_results = workflow_model.pipeline(
    pipeline_backend="local",
    run_backend="local",
    processor="noop"
)
print(f"   ✓ Pipeline: {'Success' if pipeline_results['success'] else 'Failed'}")

print("\n=== Workflow Complete ===")

## Summary

This notebook demonstrated the key features of rompy's backend system:

1. **Run Backends**: Execute models in different environments (local, Docker)
2. **Postprocessors**: Process model outputs with custom logic
3. **Pipeline Backends**: Orchestrate complete workflows
4. **Extensibility**: Easy to create and register custom backends
5. **Error Handling**: Clear error messages for debugging

The entry point-based architecture makes it easy to extend rompy with custom backends without modifying the core library. Simply implement the required interface and register your backend through Python entry points.

In [ ]:
# Cleanup: Remove temporary files
import shutil

try:
    shutil.rmtree(output_dir)
    print(f"Cleaned up temporary directory: {output_dir}")
except Exception as e:
    print(f"Could not clean up {output_dir}: {e}")